### Installation des dépendances

In [107]:
%pip install numpy
%pip install opencv-python
%pip install -U matplotlib
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import math
import os
import time

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\loicn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\loicn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\loicn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [108]:
# Define Constants
STEP_TO_NEXT_FRAME = 4 
""" in each video, we will read 1 frame every 24/STEP_TO_INDEX frames"""
DEBUG = False # if True, print debug information
NUMBER_OF_VIDEO = 100 # number of videos to index
MAX_FRAME_INDEXATION_TIME = 3 # in seconds
MAX_SEARCH_TIME = 0.1 # in seconds
VIDEOS_SIZE_ON_DISK = 444039168  # in octets
MAX_FRAME_NUMBER = 37895 # maximum number of frames in a video
N = int(MAX_FRAME_NUMBER/STEP_TO_NEXT_FRAME) # index Length
D = 128 # index Depth

In [109]:
#Define all global variables
imageList = []
videoList = []
frameIndexationTime = [] # Time must be in seconds
imageSearchTime = [] # Time must be in seconds
imageCompressionRateList = []
videoCompressionRateList = []
indexationTable = [] # Table of indexation
Descriptor_list = np.empty((N, D * 3), dtype=np.float32) # Descriptor of the image

### Lecture des fichiers

In [110]:
imagePath = "/data/jpeg/"
videoPath = "/data/mp4/"
currDirectory = os.getcwd()
imageList = os.listdir(currDirectory + imagePath)
videoList = os.listdir(currDirectory + videoPath)
# for file in imageList:
#     filePath = currDirectory + dataPath + file
#     img = cv.imread(filePath, cv.IMREAD_COLOR)

In [111]:
def getVideoParameter(videoPath:str):
    """return the frame rate and the number of frames of the video."""
    cap = cv.VideoCapture(videoPath)
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_count = cap.get(cv.CAP_PROP_FRAME_COUNT)
    return fps, frame_count
    

In [112]:
def displayRGBHistogram(histogram_r, histogram_g, histogram_b):
    plt.plot(histogram_b, color='blue', label='Canal bleu')
    plt.plot(histogram_g, color='green', label='Canal vert')
    plt.plot(histogram_r, color='red', label='Canal rouge')
    plt.xlabel('Intensité des pixels')
    plt.ylabel('Nombre de pixels')
    plt.title('Histogramme de l\'image')
    plt.legend()
    plt.show()

In [113]:

def convertImageToVector(imagePath, needToRead:bool = True):
    """return the histogram of the image."""
    if needToRead:
        image = cv.imread(imagePath)
    else:
        image = imagePath

    #Separate the color channels
    canal_b = image[:, :, 0]
    canal_g = image[:, :, 1]
    canal_r = image[:, :, 2]

    # Calculate histograms for each channel
    # D means we will have 256/D values in the histogram
    histogram_b = cv.calcHist([canal_b], [0], None, [D], [0, 255])
    histogram_g = cv.calcHist([canal_g], [0], None, [D], [0, 255])
    histogram_r = cv.calcHist([canal_r], [0], None, [D], [0, 255])
    
    vector_r = histogram_r.flatten()
    vector_g = histogram_g.flatten()
    vector_b = histogram_b.flatten() 
    
    histogramme_complet = np.concatenate((vector_r, vector_g, vector_b))
    
    if DEBUG:
        displayRGBHistogram(histogram_r, histogram_g, histogram_b)

    return histogramme_complet

In [114]:
def createIndexTable():
    """Index a video by creating a list of frames and their corresponding index. and update the indexationTable variable."""
    for video in videoList:
        path = currDirectory + videoPath + video
        fps, frame_count= getVideoParameter(path)
        cap = cv.VideoCapture(path)
        startIndex = len(indexationTable)
        stopIndex = startIndex - 1 + math.floor(frame_count/STEP_TO_NEXT_FRAME)
        for i in range(startIndex, stopIndex):
            frameNumber = STEP_TO_NEXT_FRAME*(i-startIndex)
            indexationTable.append((i, video, (frameNumber/fps)))
            currImage = cap.set(cv.CAP_PROP_POS_FRAMES, frameNumber)
            ret, currImage = cap.read()
            Descriptor_list[i] = convertImageToVector(currImage, False)
        
createIndexTable()

In [115]:
def CalculateCompressionRate():
    print(len(indexationTable))
    print(Descriptor_list.shape)
    rate = 1- (3*D*N*4)/VIDEOS_SIZE_ON_DISK #*4 because we use float32 and 3 because we have 3 channels RGB
    print(rate)
    
CalculateCompressionRate()

9338
(9473, 384)
0.9672314312596856


In [116]:
def plotImage(image, type):
    figure = plt.figure(figsize = (10,10))
    imageout = np.clip(image,0,255)
    imageout= imageout.astype(type)
    plt.imshow(imageout, cmap='gray')
    plt.show()

In [127]:
def searchImage():
    bestDistance = 0
    for image in imageList:
        path = currDirectory + imagePath + image
        if DEBUG:
            plotImage(cv.cvtColor(cv.imread(path), cv.COLOR_BGR2RGB), 'uint8')
        imageVector = convertImageToVector(path)
        for i in range(len(indexationTable)):
            # calculer la norme de la différence entre l'image et la liste de descripteurs
            dist_r = np.linalg.norm(imageVector[0:D-3] - Descriptor_list[i][0:D-3])
            dist_g = np.linalg.norm(imageVector[D:2*D-1] - Descriptor_list[i][D:2*D-1])
            dist_b = np.linalg.norm(imageVector[2*D:3*D-1] - Descriptor_list[i][2*D:3*D-1])
            if i == 0:
                bestDistance_r = dist_r
                bestDistance_g = dist_g
                bestDistance_b = dist_b
                bestIndex_r = bestIndex_g= bestIndex_b=i
            if dist_r < bestDistance_r:
                bestDistance_r = dist_r
                bestIndex_r = i
            if dist_g < bestDistance_g:
                bestDistance_g = dist_g
                bestIndex_g = i
            if dist_b < bestDistance_b:
                bestDistance_b = dist_b
                bestIndex_b = i
            
            foundVideo_r = indexationTable[bestIndex_r][1]
            foundVideo_g = indexationTable[bestIndex_g][1]
            foundVideo_b = indexationTable[bestIndex_b][1]
            time_r = indexationTable[bestIndex_r][2]
            time_g = indexationTable[bestIndex_g][2]
            time_b = indexationTable[bestIndex_b][2]
            if foundVideo_r == foundVideo_g == foundVideo_b:
                bestVideo = foundVideo_r
                bestTime = (time_r+time_g+time_b)/3
            elif foundVideo_r == foundVideo_g:
                bestVideo = foundVideo_r
                bestTime = (time_r+time_g)/2
            elif foundVideo_r == foundVideo_b:
                bestVideo = foundVideo_r
                bestTime = (time_r+time_b)/2
            elif foundVideo_g == foundVideo_b:
                bestVideo = foundVideo_g
                bestTime = (time_g+time_b)/2
            else:
                bestVideo = "out"
                bestTime = ""
            
        print("The best match is the frame number ", (image.split(".")[0], bestVideo, bestTime))   
        break
    
searchImage()

The best match is the frame number  ('i000', 'v001.mp4', 5.111111111111111)
The best match is the frame number  ('i001', 'v050.mp4', 16.850166666666667)
The best match is the frame number  ('i002', 'out', '')
The best match is the frame number  ('i003', 'v042.mp4', 12.723822222222223)
The best match is the frame number  ('i004', 'v075.mp4', 1.4400000000000002)
The best match is the frame number  ('i005', 'v048.mp4', 9.84)
The best match is the frame number  ('i006', 'v042.mp4', 4.048488888888889)
The best match is the frame number  ('i007', 'out', '')
The best match is the frame number  ('i008', 'out', '')
The best match is the frame number  ('i009', 'v064.mp4', 10.143466666666667)
The best match is the frame number  ('i010', 'v059.mp4', 9.4)
The best match is the frame number  ('i011', 'v001.mp4', 7.0)
The best match is the frame number  ('i012', 'v062.mp4', 12.0)
The best match is the frame number  ('i013', 'v016.mp4', 6.673333333333334)
The best match is the frame number  ('i014', '

KeyboardInterrupt: 

In [118]:
def writeSearchResults():
    

SyntaxError: incomplete input (3781726569.py, line 2)

In [ ]:
def verifyFrameIndexationTime():
    """Function to verify if the indexation time of each frame is less than the max time allowed for indexation per frame."""
    videoToReindex = []
    totalIndexationTime = 0
    for i in range(0, len(frameIndexationTime)):
        totalIndexationTime += frameIndexationTime[i]
        if(frameIndexationTime[i] > MAX_FRAME_INDEXATION_TIME):
            videoToReindex.append(i)
    if len(videoToReindex) != 0:
        print("Les vidéos suivantes doivent être réindexées car elles prennent plus de temps que le temps maximal d'indexation par frame: " + str(MAX_FRAME_INDEXATION_TIME) + " secondes")
        for i in range(0, len(videoToReindex)):
            print(videoList[videoToReindex[i]])
    print("Toutes les vidéos ont été correctement indexées\nLe temps d'indexation total est: " + str(totalIndexationTime/60.0) + " minutes")


In [ ]:
def verifyImageSearchTime():
    """Function to verify if the search time for each image is less than the max time allowed for search per image."""
    imageToResearch = []
    totalSearchTime = 0
    for i in range(0, len(imageSearchTime)):
        totalSearchTime += imageSearchTime[i]
        if(imageSearchTime[i] > MAX_SEARCH_TIME):
            imageToResearch.append(i)
    if len(imageToResearch) != 0:
        print("La recherche doit être révisée pour les images suivantes car elles prennent plus de temps que le temps maximal de recherche par image: " + str(MAX_SEARCH_TIME) + " secondes")
        for i in range(0, len(imageToResearch)):
            print(imageList[imageToResearch[i]])
    print("Le temps de recherche total est: " + str(totalSearchTime) + " secondes")

In [ ]:
# Cell for test purposes
calculateTotalFrame()